In [1]:
import threading
import time

import json
import numpy as np
import pandas as pd
# import requests
import time
import re

from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# from tqdm import tqdm
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

In [2]:
executable_path = 'C:/Users/Manych/chromium/chromedriver.exe'

In [3]:
with open('./Project_7_data/marks_models_for_parsing.json') as f: 
    marks_models_for_parsing = json.loads(f.read())
# model_generation_year = pd.read_csv('./Project_7_data/model_generation_year.csv')

In [5]:
# marks_models_for_parsing

# ===================

In [18]:
def get_features_from_ticket(ticket_url, driver):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service : обьект selenium.webdriver.chrome.service
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
#     driver = webdriver.Chrome(service=service)
#     driver.maximize_window()
    driver.get(ticket_url)
    # Находим и кликаем 'Все опции'
    try:
        butt = driver.find_element(By.CLASS_NAME, 'ComplectationGroupsDesktop__cut') 
        butt.click() 
    except Exception: pass
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
#     driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  

    # проверка корректности результата BeautifulSoup
    if ticket_bs:
    # получение признаков из карточки текущего обьявления
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN

        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[2].text.strip().upper()
        except Exception: features['brand'] = np.NaN
            
        # car_url
        features['car_url'] = ticket_url
            
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN 
            
        # description
        try:
            rows = ticket_bs.find('div',class_='CardDescriptionHTML').find_all('span')
            features['description'] = '\n'.join([row.text for row in rows])
        except Exception: features['description'] = np.NaN
        
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
            
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN            

        # equipment_dict 
        try: 
            complectation_groups = ticket_bs.find(
                'div',class_='ComplectationGroupsDesktop__row').find_all(
                'div',class_='ComplectationGroupsDesktop__group')
            features['equipment_dict'] = {
                group.text.split('•')[0]: group.text.split('•')[1:] for group in complectation_groups}
        except Exception: features['equipment_dict'] = np.NaN             
  
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        
        except Exception: features['fuel_type'] = np.NaN
            
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN            
            
        # model_name
        brand = features['brand']
        model_name_start = ticket_url.find(brand.lower()) + len(brand) + 1
        model_name_end = ticket_url.find('/',model_name_start)
        features['model_name'] = ticket_url[model_name_start:model_name_end].upper()        
#         try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[3].text.strip()
#         except Exception: features['model_name'] = np.NaN            
            
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN

        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
            
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
            
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN            
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'  

        # Владение
        
        try: features['Владение'] = ticket_bs.find('li',class_='CardInfoRow_owningTime').find_all('span')[1].text
        except Exception: features['Владение'] = np.NaN             
            
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN            
            
        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN               
            
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN            
            
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN         
            
        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN

        # modelDate              
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[2].text.strip() + ' ' +
                                     modelDate_tag[3].text.strip() + ' ' +
                                     modelDate_tag[4].text.strip())  
        except Exception: features['modelDate'] = np.NaN   
    return features

In [107]:
def get_features_from_ticket_2(ticket_url):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
    service : обьект selenium.webdriver.chrome.service
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    driver = webdriver.Chrome(executable_path,options=option)
#     driver.maximize_window()
    driver.get(ticket_url)
    # Находим и кликаем 'Все опции'
    try:
        butt = driver.find_element(By.CLASS_NAME, 'ComplectationGroupsDesktop__cut') 
        butt.click() 
    except Exception: pass
    # Получаем содержимое html-страницы:
    ticket_res = driver.execute_script("return document.body.innerHTML;")
    # Закрываем процесс браузера:
    driver.quit()
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res, 'html.parser')  # 'lxml'

    # проверка корректности результата BeautifulSoup
    if ticket_bs:
    # получение признаков из карточки текущего обьявления
        # bodyType      
        try: features['bodyType'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN

        # brand  
        try: features['brand'] = ticket_url.split('/')[-4].upper()
#         try: features['brand'] = ticket_bs.find_all(
#             'a',class_='CardBreadcrumbs__itemText')[2].text.strip().upper()
        except Exception: features['brand'] = np.NaN
            
        # car_url
        features['car_url'] = ticket_url
            
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN 
            
        # description
        try:
            rows = ticket_bs.find('div',class_='CardDescriptionHTML').find_all('span')
            features['description'] = '\n'.join([row.text for row in rows])
        except Exception: features['description'] = np.NaN
        
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
            
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN            

        # equipment_dict 
        try: 
            complectation_groups = ticket_bs.find(
                'div',class_='ComplectationGroupsDesktop__row').find_all(
                'div',class_='ComplectationGroupsDesktop__group')
            features['equipment_dict'] = {
                group.text.split('•')[0]: group.text.split('•')[1:] for group in complectation_groups}
        except Exception: features['equipment_dict'] = np.NaN             
  
        # fuelType
        try: features['fuelType'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        
        except Exception: features['fuel_type'] = np.NaN
            
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN            
            
        # model_name
        try: features['model_name'] = ticket_url.split('/')[-3].upper()
        except Exception: features['model_name'] = np.NaN            
            
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN

        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
            
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
            
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN            
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'  

        # Владение
        
        try: features['Владение'] = ticket_bs.find('li',class_='CardInfoRow_owningTime').find_all('span')[1].text
        except Exception: features['Владение'] = np.NaN             
            
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN            
            
        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN               
            
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN            
            
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN         
            
        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN

        # modelDate              
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[2].text.strip() + ' ' +
                                     modelDate_tag[3].text.strip() + ' ' +
                                     modelDate_tag[4].text.strip())  
        except Exception: features['modelDate'] = np.NaN   
    return features

# ===================

In [75]:
from joblib import Parallel, delayed
from project_7_constants import *

In [76]:
# qwerty = pd.DataFrame(columns=['asd','sdf','dfg','fgh'])

# def heavy(x,y):
#     # имитации операций ввода-вывода
#     res = pd.Series([x*y*1,x*y*2,x*y*3,x*y*4],index = ['asd','sdf','dfg','fgh'])
#     time.sleep(2)
#     return res

In [77]:
# %%time
# y = 1
# page_items = Parallel(n_jobs=7, prefer="threads")(
#     delayed(heavy)(i,y) for i in range(1,10))
# qwerty.append(page_items)

In [78]:
# {k: marks_models_for_parsing[k][:3] for k in ['VOLKSWAGEN']}

In [111]:
train = pd.DataFrame(columns=data_columns)

In [112]:
%%time
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
option.add_argument('headless')

for mark in {k: marks_models_for_parsing[k] for k in ['VOLKSWAGEN']}: # 
    print(mark)
    for model in marks_models_for_parsing[mark][:4]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/' + '?output_type=table'
        driver = webdriver.Chrome(executable_path, options=option)
#         driver.maximize_window()
    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,99): 
            if pages_num==1: page_url = model_url
            else:            page_url = model_url + '&page=' +  str(pages_num)  
            # получем html страницы
#             driver = webdriver.Chrome(executable_path)
#             driver.maximize_window()
            driver.get(page_url)
            page_html = driver.execute_script("return document.body.innerHTML;")
            # создаем обьект bs4.BeautifulSoup из html страницы
            page_bs = BeautifulSoup(page_html, 'lxml') # 'html.parser'
            # список html-ек карточек на странице                                               
            tickets_on_page = page_bs.find_all('a', class_='ListingItemTitle__link') 
            urls_on_page = [x.get('href') for x in tickets_on_page]
#             print (len(urls_on_page))
            # выход по исчерпанию страниц текущей модели текущей марки
            if not tickets_on_page: 
                driver.quit()
                break            
#             # обработка карточек на странице                
#             page_items = Parallel(n_jobs=10, prefer="threads")(
#                 delayed(get_features_from_ticket)(ticket_url,driver) for ticket_url in urls_on_page)
            page_items = Parallel(n_jobs=-1, prefer='threads')(
                delayed(get_features_from_ticket_2)(ticket_url) for ticket_url in urls_on_page)
            train = train.append(page_items)
#             for ticket in tickets_on_page:
#                 # получаем url карточки текущего обьявления
#                 ticket_url = ticket.get('href')
#                 # извлекаем признаки и заполняем строку train-а
#                 train.loc[len(train)] =get_features_from_ticket(ticket_url,driver)
#                 time.sleep(0.5) 
#             time.sleep(0.5) 
#             driver.quit()
        driver.quit()
        print ('|',len(train),end='') # <==============================             
    print ('|') # <============================== 

VOLKSWAGEN
| 26| 35| 50| 69|
Wall time: 2min 4s


In [73]:
# ticket_url = 'https://auto.ru/cars/used/sale/volkswagen/amarok/1114951157-8e6265a1/'
# ticket_url.split('/')[-3].upper()

In [71]:
train

,bodyType,bodytype,brand,car_url,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,offerprice,productionDate,sell_id,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль
0,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,белый,"Автомобиль в отличном состоянии, вложения не т...",2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,1650000,2013,1114677051,механическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый
1,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серебристый,АВТОРАССРОЧКА НА АВТОМОБИЛИ С ПРОБЕГОМ 0%!\n\n...,3.0,204,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,3039000,2016,1115036990,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый
2,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,коричневый,ЛОТ: 01299790\nЛокация №19\nФаворит Хофф\n\n\n...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,1869000,2013,1105973842,механическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый
3,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,чёрный,ЛОТ: 01316216\nЛокация №8\nFAVORIT MOTORS Peug...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,2900000,2015,1114844447,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый
4,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серый,Продажа от официального дилера ГК «АВТОМИР».\n...,2.0,180,{'Безопасность': ['Подушки безопасности боковы...,Дизель,...,3420000,2016,1115042580,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый
5,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серый,ЛОТ: 01307365\nЛокация №19\nФаворит Хофф\n\n\n...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,2700000,2016,1106543020,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый
6,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,белый,Volkswagen Amarok I Пикап двойная кабина full-...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,1639500,2013,1106041615,механическая,EUROPEAN,2 владельца,8 лет,Оригинал,полный,Левый
7,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,коричневый,АВТОРАССРОЧКА НА АВТОМОБИЛИ С ПРОБЕГОМ 0%!*\n\...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,1663000,2012,1114836194,механическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый
8,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серебристый,Volkswagen Amarok I Пикап двойная кабина full-...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,1629500,2012,1102331043,автоматическая,EUROPEAN,1 владелец,8 лет и 12 месяцев,Оригинал,полный,Левый
9,пикап двойная кабина,NaN,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/amar...,серый,Если Вы хотите приобрести автомобиль с пробего...,2.0,180,{'Безопасность': ['Подушка безопасности водите...,Дизель,...,4297000,2020,1115094448,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый


In [92]:
page_items = Parallel(n_jobs=10, prefer="threads")(
    delayed(get_features_from_ticket)(ticket_url,driver) for ticket_url inurls_on_page)
qwerty.append(page_items)

Wall time: 4.04 s


,asd,sdf,dfg,fgh
0,1,2,3,4
1,2,4,6,8
2,3,6,9,12
3,4,8,12,16
4,5,10,15,20
5,6,12,18,24
6,7,14,21,28
7,8,16,24,32
8,9,18,27,36
9,10,20,30,40


In [ ]:
# option = webdriver.ChromeOptions()
# chrome_prefs = {}
# option.experimental_options["prefs"] = chrome_prefs
# chrome_prefs["profile.default_content_settings"] = {"images": 2}
# chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
# PATH = 'C:\Program Files (x86)\chromedriver.exe'
# browser = webdriver.Chrome(executable_path = PATH, options=option)
# browser.get('https://www.yahoo.com/')
# browser.find_element_by_xpath('//*[@class="btn primary"]').click()